<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/concept_implementation/continuous_gaussian_propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openfermion
!pip install "StrawberryFields==0.12.1"

# **Gaussian propagation function**

In [47]:
#@title Auxilary
from itertools import groupby

import numpy as np

from openfermion.utils import is_hermitian


def quadratic_coefficients(operator):
    r"""Return the quadratic coefficient matrix representing a Gaussian Hamiltonian.

    A Gaussian Hamiltonian is any combination of quadratic operators
    that can be written in quadratic form:

    .. math:: H = \frac{1}{2}\mathbf{r}^T A\mathbf{r} + \mathbf{r}^T \mathbf{d}

    where :math:`A\in\mathbb{R}^{2N\times 2N}` is a symmetric matrix,
    :math:`\mathbf{d}\in\mathbb{R}^{2N}` is a real vector, and
    :math:`\mathbf{r} = (\x_1,\dots,\x_N,\p_1,\dots,\p_N)` is the vector
    of quadrature operators in :math:`xp`-ordering.

    This function accepts a bosonic Gaussian Hamiltonian, and returns the
    matrix :math:`A` and vector :math:`\mathbf{d}` representing the
    quadratic and linear coefficients.

    Args:
        operator (QuadOperator): a bosonic Gaussian Hamiltonian
    Returns:
        tuple (A, d): a tuple contains a 2Nx2N real symmetric numpy array,
        and a length-2N real numpy array, where N is the number of modes
        the operator acts on.
    """
    if not operator.is_gaussian():
        raise ValueError("Hamiltonian must be Gaussian "
                         "(quadratic in the quadrature operators).")

    if not is_hermitian(operator):
        raise ValueError("Hamiltonian must be Hermitian.")

    num_modes = max([op[0] for term in operator.terms for op in term])+1
    A = np.zeros([2*num_modes, 2*num_modes])
    d = np.zeros([2*num_modes])
    for term, coeff in operator.terms.items():
        c = coeff.real
        if len(term) == 2:
            if term[0][1] == term[1][1]:
                if term[0][1] == 'q':
                    A[term[0][0], term[1][0]] = c
                elif term[0][1] == 'p':
                    A[num_modes+term[0][0], num_modes+term[1][0]] = c
            else:
                if term[0][1] == 'q':
                    A[term[0][0], num_modes+term[1][0]] = c
                elif term[0][1] == 'p':
                    A[num_modes+term[0][0], term[1][0]] = c
        elif len(term) == 1:
            if term[0][1] == 'q':
                d[num_modes+term[0][0]] = -c
            elif term[0][1] == 'p':
                d[term[0][0]] = c

    A += A.T
    return A, d



class BoseHubbardError(ValueError):
    """Custom error function for invalid Bose-Hubbard Hamiltonians."""

    def with_traceback(self, tb): #pragma: no cover
        # pylint: disable=useless-super-delegation
        """This method sets argument ``tb`` as the new traceback for the exception
        and returns the exception object. See the
        `Python documentation <https://docs.python.org/3/library/exceptions.html#BaseException.with_traceback>`_
        for more details.
        """
        # this method is overwritten simply due to a bug in Sphinx,
        # which automatically pulls this method into the documentation
        # even if it is not present. By overwriting, we at least get
        # to modify the docstring presented.
        return super().with_traceback(tb)



def extract_tunneling(H):
    """Extracts the tunneling terms from Bose-Hubbard Hamiltonians.

    Args:
        H (BosonOperator): A Bose-Hubbard Hamiltonian.

    Returns:
        list: Returns a length-2 list of the form ``[[(i, j),...], t]``
        where ``[(i, j),...]`` is a list containing pairs of modes
        that are entangled by beamsplitters due to tunneling,
        and ``t`` (float) is the tunneling coefficient.
    """
    BS = {}

    # created a sorted list containing terms with two operators,
    # indexed by the mode of each operator
    tmp = sorted([((o[0][0], o[1][0]), o, c) for o, c in H.terms.items() if len(o) == 2])

    # group into terms
    for key, group in groupby(tmp, lambda x: x[0]):

        # consider only multi-mode terms
        if key[0] != key[1]:
            # copy iterable to a list
            group_list = list(group)

            # check only one
            if len(group_list) != 2:
                raise BoseHubbardError

            # check ladders are of the right form
            # of bi^bj and bj^bi
            ladders = np.array(list(np.array(group_list)[:, 1]))[:, :, 1]
            ladders = set([tuple(i) for i in ladders.tolist()])
            if ladders != {(0, 1), (1, 0)}:
                raise BoseHubbardError

            # check coefficients are the same
            if group_list[0][-1] != group_list[1][-1]:
                raise BoseHubbardError

            BS[key] = group_list[0][-1]
            t = BS[key]

    # check all beamsplitters have the same tunneling coefficient
    if len(set(BS.values())) != 1:
        raise BoseHubbardError

    # check t is real
    if t.imag != 0:
        raise BoseHubbardError

    return [list(BS.keys()), -t.real]



def extract_onsite_chemical(H):
    """Extracts the onsite interactions and chemical potential terms
    from Bose-Hubbard Hamiltonians.

    Args:
        H (BosonOperator): A Bose-Hubbard Hamiltonian.

    Returns:
        tuple(list, list): Returns a tuple containing two lists; the
        first list is the onsite interaction, and the second list is
        the chemical potential.
        Each list is a length-2 list of the form ``[[i,j,...], t]``
        where ``[i,j,...]`` is a list containing modes operated on,
        and t is the onsite coefficient U or chemical potential mu.
    """
    # Note: we have to consider both onsite and chemical potential terms
    # at the same time, since they share the same structure.
    onsite = {}
    chemical = {}
    mu = 0
    U = 0

    # created a sorted list containing terms where the first two operators
    # act on the *same* mode. It is indexed by this mode.
    tmp = sorted([(o[0][0], o, c) for o, c in H.terms.items() if o[0][0] == o[1][0]])

    # remove multi mode terms
    tmp = sorted([i for i in tmp if len(set(np.array(i[1]).T[0])) == 1])

    # iterate through elements grouped by the indexed mode
    for key, group in groupby(tmp, lambda x: x[0]):

        # copy iterable to a list
        group_list = list(group)

        # check correct number of terms
        # should be one bk^ bk and one bk^ bk bk^ bk if onsite is present,
        # or just one bk^ bk for chemical potential only.
        if len(group_list) > 2:
            raise BoseHubbardError

        if len(group_list) == 1 and len(group_list[0][1]) != 2:
            raise BoseHubbardError

        coeff_dict = {}

        for term in group_list:
            # check correct ordering of ladder operators
            tmp2 = np.array(term[1])[:, 1]
            expected = np.zeros(tmp2.shape)
            expected[::2] = 1
            if not np.all(tmp2 == expected):
                raise BoseHubbardError

            # if the ordering is correct, save the coefficient,
            # with the key as the length of the term (2 or 4).
            coeff_dict[tmp2.shape[0]] = term[-1]

        # check coefficients are of the right form
        term_lengths = sorted(list(coeff_dict.keys()))
        if term_lengths == [2, 4]:
            # on-site interactions present
            U = coeff_dict[4]*2
            onsite[key] = U

            # chemical potential present
            mu = -coeff_dict[2]-onsite[key]/2
            chemical[key] = mu

        elif term_lengths == [2]:
            # only chemical potential present
            mu = -coeff_dict[2]
            chemical[key] = mu
        else:
            raise BoseHubbardError

    # check all have the same U and mu
    if len(set(onsite.values())) > 1:
        raise BoseHubbardError
    if len(set(chemical.values())) > 1:
        raise BoseHubbardError

    onsite = [list(onsite.keys()), U] if onsite else []

    if mu == 0:
        chemical = []
    else:
        chemical = [list(chemical.keys()), mu] if chemical else []

    return onsite, chemical



def extract_dipole(H):
    """Extracts the dipole terms from Bose-Hubbard Hamiltonians.

    Args:
        H (BosonOperator): A Bose-Hubbard Hamiltonian.

    Returns:
        list: Returns a length-2 list of the form ``[[(i, j),...], V]``
        where ``[(i, j),...]`` is a list containing pairs of modes
        that are entangled due to nearest-neighbour interactions,
        and ``V`` (float) is the dipole coefficient.
    """
    dipole = {}

    # extract all length 4 multimode terms
    tmp = [term for term in H.terms.items() if len(term[0]) == 4]

    # created a sorted list containing terms where the first operator
    # and the third operator act on the *same* mode. It is indexed by these modes.
    tmp = sorted([((o[0][0], o[2][0]), o, c) for o, c in tmp if o[0][0] != o[2][0]])

    if not tmp:
        return []

    # iterate through elements grouped by the indexed modes
    for key, group in groupby(tmp, lambda x: x[0]):
        # copy iterable to a list
        group_list = list(group)

        # check only one term is present
        if len(group_list) != 1:
            raise BoseHubbardError

        # check ladders are of the right form bi^bi bj^ bj
        ladders = np.array(list(np.array(group_list)[:, 1]))[:, :, 1]
        ladders = set([tuple(i) for i in ladders.tolist()])
        if ladders != {(1, 0, 1, 0)}:
            raise BoseHubbardError

        # extract the dipole coefficient
        V = group_list[0][-1]
        dipole[key] = V

    # check all terms have the same V
    if len(set(dipole.values())) != 1:
        raise BoseHubbardError

    return [list(dipole.keys()), V]



def trotter_layer(H, t, k):
    """Returns a single Trotter layer for a Bose-Hubbard Hamiltonian.

    Args:
        H (BosonOperator): A Bose-Hubbard Hamiltonian.
        t (float): the time propagation duration.
        k (int): the number of products in the truncated
            Lie product formula.

    Returns:
        dict: A dictionary containing the items:
            * ``'BS': (theta, phi, modes)`` corresponding to beamsplitters
              with parameters ``(theta, phi)`` acting on the list of modes provided.
            * ``'K': (kappa, modes)`` corresponding to Kerr gates with parameter
              ``kappa`` acting on the list of modes provided.
            * ``'R': (r, modes)`` corresponding to rotation gates with parameter
              ``r`` acting on the list of modes provided.
            * ``'CK': (kappa, modes)`` corresponding to a cross-Kerr interactions with parameter
              ``kappa`` acting on the list of modes provided.
    """
    try:
        BS = extract_tunneling(H)
        onsite, chemical = extract_onsite_chemical(H)
        dipole = extract_dipole(H)
    except BoseHubbardError:
        raise BoseHubbardError("Hamiltonian is not a valid Bose-Hubbard model. "
                               "Are you generating it using the bose_hubbard function?")

    layer_dict = {}

    theta = -t*BS[1]/k
    phi = np.pi/2
    layer_dict['BS'] = (theta, phi, BS[0])

    U = 0
    if onsite:
        U = onsite[1]
        kappa = -t*U/(2*k)
        layer_dict['K'] = (kappa, onsite[0])

    mu = 0
    if chemical:
        mu = chemical[1]
        r = t*(0.5*U+mu)/(2*k)
        layer_dict['R'] = (r, chemical[0])
    elif onsite:
        layer_dict['R'] = (t*U/(2*k), onsite[0])

    if dipole:
        V = dipole[1]
        layer_dict['CK'] = (-t*V/k, dipole[0])

    return layer_dict

In [48]:
#@title GaussianPropagation
import sys

import numpy as np
from scipy.linalg import expm, inv

from openfermion.ops import QuadOperator, BosonOperator
from openfermion.transforms import get_quad_operator, get_boson_operator
from openfermion.utils import is_hermitian
from openfermion.transforms import prune_unused_indices

import strawberryfields as sf
import strawberryfields.program_utils as pu
from strawberryfields.ops import (BSgate,
                                  CKgate,
                                  Decomposition,
                                  GaussianTransform,
                                  Kgate,
                                  Rgate,
                                  Xgate,
                                  Zgate)
from strawberryfields.program_utils import Command
from strawberryfields.backends.shared_ops import sympmat

from strawberryfields.circuitspecs import GaussianSpecs, FockSpecs, TFSpecs

class GaussianPropagation(Decomposition):
    r"""Propagate the specified qumodes by a bosonic Gaussian Hamiltonian.

    A Gaussian Hamiltonian is any combination of quadratic operators
    that can be written in quadratic form:

    .. math:: H = \frac{1}{2}\mathbf{r}^T A\mathbf{r} + \mathbf{r}^T \mathbf{d}

    where:

    * :math:`A\in\mathbb{R}^{2N\times 2N}` is a symmetric matrix,
    * :math:`\mathbf{d}\in\mathbb{R}^{2N}` is a real vector, and
    * :math:`\mathbf{r} = (\x_1,\dots,\x_N,\p_1,\dots,\p_N)` is the vector
      of quadrature operators in :math:`xp`-ordering.

    This operation calculates the corresponding Gaussian symplectic
    transformation via the following relation:

    .. math:: S = e^{\Omega A t}

    where

    .. math::
        \Omega=\begin{bmatrix}0&I_N\\-I_N&0\end{bmatrix}\in\mathbb{R}^{2N\times 2N}

    is the symplectic matrix.

    Depending on whether the resulting symplectic transformation is passive
    (energy preserving) or active (non-energy preserving), the Clements or
    Bloch-Messiah decomposition in Strawberry Fields is then used to decompose
    the Hamiltonian into a set of CV gates.

    Args:
        operator (BosonOperator, QuadOperator): a bosonic Gaussian Hamiltonian
        t (float): the time propagation value. If not provided, default value is 1.
        mode (str): By default, ``mode='local'`` and the Hamiltonian is assumed to apply to only
            the applied qumodes (q[i], q[j],...). For instance, a_0 applies to q[i], a_1 applies to q[j].
            If instead ``mode='global'``, the Hamiltonian is instead applied to the entire register,
            i.e., a_0 applies to q[0], applies to q[1], etc.
    """
    ns = None
    def __init__(self, operator, t=1, mode='local'):
        super().__init__([t])

        if not is_hermitian(operator):
            raise ValueError("Hamiltonian must be Hermitian.")

        if mode == 'local':
            quad_operator = prune_unused_indices(operator)
        elif mode == 'global':
            quad_operator = operator

        if isinstance(quad_operator, BosonOperator):
            quad_operator = get_quad_operator(quad_operator, hbar=sf.hbar)

        A, d = quadratic_coefficients(quad_operator)

        if mode == 'local':
            self.ns = A.shape[0]//2
        elif mode == 'global':
            # pylint: disable=protected-access
            self.ns = pu.Program_current_context.num_subsystems
            if A.shape[0] < 2*self.ns:
                # expand the quadratic coefficient matrix to take
                # into account the extra modes
                A_n = A.shape[0]//2
                tmp = np.zeros([2*self.ns, 2*self.ns])

                tmp[:A_n, :A_n] = A[:A_n, :A_n]
                tmp[:A_n, self.ns:self.ns+A_n] = A[:A_n, A_n:]
                tmp[self.ns:self.ns+A_n, :A_n] = A[A_n:, :A_n]
                tmp[self.ns:self.ns+A_n, self.ns:self.ns+A_n] = A[A_n:, A_n:]

                A = tmp

        self.S = expm(sympmat(self.ns) @ A * t)

        self.disp = False
        if not np.all(d == 0.):
            self.disp = True
            if np.all(A == 0.):
                self.d = d*t
            else:
                if np.linalg.cond(A) >= 1/sys.float_info.epsilon:
                    # the matrix is singular, add a small epsilon
                    eps = 1e-9
                    epsI = eps * np.identity(2*self.ns)
                    s = inv(A+epsI) @ d
                    tmp = (np.identity(2*self.ns) \
                        - expm(sympmat(self.ns) @ (A+epsI) * t).T) @ s / eps
                else:
                    s = inv(A) @ d
                    tmp = s - self.S.T @ s

                self.d = np.zeros([2*self.ns])
                self.d[self.ns:] = tmp[:self.ns]
                self.d[:self.ns] = tmp[self.ns:]

    def decompose(self, reg):
        """Return the decomposed commands"""
        cmds = []
        cmds += [Command(GaussianTransform(self.S), reg)]
        if self.disp:
            cmds += [Command(Xgate(x), reg) for x in self.d[:self.ns] if x != 0.]
            cmds += [Command(Zgate(z), reg) for z in self.d[self.ns:] if z != 0.]
        return cmds



class BoseHubbardPropagation(Decomposition):
    r"""Propagate the specified qumodes by a Bose-Hubbard Hamiltonian.

    The Bose-Hubbard Hamiltonian has the form

    .. math::
        H = -J\sum_{i=1}^N\sum_{j=1}^N A_{ij} \ad_i\a_j
            + \frac{1}{2}U\sum_{i=1}^N \a_i^\dagger \a_i (\ad_i \a_i - 1)
            - \mu \sum_{i=1}^N \ad_i \a_i
            + V \sum_{i=1}^N\sum_{j=1}^N A_{ij} \ad_i \a_i \ad_j \a_j.

    where:

    * :math:`A` is a real symmetric matrix of ones and zeros defining the adjacency of
      each pairwise combination of nodes :math:`(i,j)` in the :math:`N`-node system,
    * :math:`J` represents the transfer integral or hopping term of the boson between nodes,
    * :math:`U` is the on-site interaction potential,
    * :math:`\mu` is the chemical potential,
    * :math:`V` is the dipole-dipole or nearest neighbour interaction term.

    BoseHubbard Hamiltonians can be generated using the BosonOperator manually, or
    on a (periodic/non-peridic) two-dimensional lattice via the function
    ``openfermion.hamiltonians.bose_hubbard``
    (see the `OpenFermion documentation <http://openfermion.readthedocs.io/en/latest/openfermion.html#openfermion.hamiltonians.bose_hubbard>`_).

    In Strawberry Fields, the Bose-Hubbard propagation is performed by applying the
    Lie-product formula, and decomposing the unitary operations into a combination
    of beamsplitters, Kerr gates, and phase-space rotations.

    Args:
        operator (BosonOperator, QuadOperator): a bosonic Gaussian Hamiltonian
        t (float): the time propagation value. If not provided, default value is 1.
        k (int): the number of products in the truncated Lie product formula.
        mode (str): By default, ``mode='local'`` and the Hamiltonian is assumed to apply to only
            the applied qumodes (q[i], q[j],...). For instance, a_0 applies to q[i], a_1 applies to q[j].
            If instead ``mode='global'``, the Hamiltonian is instead applied to the entire register,
            i.e., a_0 applies to q[0], applies to q[1], etc.
    """
    ns = None
    def __init__(self, operator, t=1, k=20, mode='local'):
        super().__init__([t])

        if not is_hermitian(operator):
            raise ValueError("Hamiltonian must be Hermitian.")

        if (not isinstance(k, int)) or k <= 0:
            raise ValueError("Argument k must be a postive integer.")

        if mode == 'local':
            boson_operator = prune_unused_indices(operator)
        elif mode == 'global':
            boson_operator = operator

        if isinstance(boson_operator, QuadOperator):
            boson_operator = get_boson_operator(boson_operator, hbar=sf.hbar)

        self.layer = trotter_layer(boson_operator, t, k)
        self.num_layers = k

        num_modes = max([op[0] for term in operator.terms for op in term])+1

        if mode == 'local':
            self.ns = num_modes
        elif mode == 'global':
            # pylint: disable=protected-access
            self.ns = pu.Program_current_context.num_subsystems

    def decompose(self, reg):
        # make BS gate
        theta = self.layer['BS'][0]
        phi = self.layer['BS'][1]
        BS = BSgate(theta, phi)

        # make cross-Kerr gate
        CK = None
        param = self.layer.get('CK', [0])[0]
        if param != 0:
            CK = CKgate(param)

        # make Kerr gate
        K = None
        param = self.layer.get('K', [0])[0]
        if param != 0:
            K = Kgate(param)

        # make rotation gate
        R = None
        param = self.layer.get('R', [0])[0]
        if param != 0:
            R = Rgate(param)

        cmds = []

        for i in range(self.num_layers): #pylint: disable=unused-variable
            for q0, q1 in self.layer['BS'][2]:
                cmds.append(Command(BS, (reg[q0], reg[q1])))

            if CK is not None:
                for q0, q1 in self.layer['CK'][1]:
                    cmds.append(Command(CK, (reg[q0], reg[q1])))

            if K is not None:
                for mode in self.layer['K'][1]:
                    cmds.append(Command(K, reg[mode]))

            if R is not None:
                for mode in self.layer['R'][1]:
                    cmds.append(Command(R, reg[mode]))

        return cmds
  
GaussianSpecs.decompositions.update({"GaussianPropagation": {}})
FockSpecs.decompositions.update({"BoseHubbardPropagation": {}, "GaussianPropagation": {}})
TFSpecs.decompositions.update({"BoseHubbardPropagation": {}, "GaussianPropagation": {}})

In [49]:
#@title Capturing
from io import StringIO 
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

# **Problem hamiltonian**

In [91]:
import strawberryfields as sf
from strawberryfields.ops import *
from openfermion.ops import QuadOperator

Form problem hamiltonian using gaussian propagation <br><br>
$$f(x) = (x-10^{-8})^2$$


In [92]:
target = 1e-8
H = QuadOperator('q0 q0', 1.0) - QuadOperator('q0', target) 
print(H)

-1e-08 [q0] +
1.0 [q0 q0]


In [93]:
prog = sf.Program(1)
with prog.context as q:
    GaussianPropagation(H, 1/1000) | q[0]

In [94]:
eng = sf.Engine("gaussian")
state = eng.run(prog).state
with Capturing() as hamiltonian: 
  eng.print_applied()
  
hamiltonian = hamiltonian[1:]
for gate in hamiltonian:
  print(gate)

Rgate(0.7849) | (q[0])
Sgate(-0.001, 0) | (q[0])
Rgate(-0.7859) | (q[0])
Dgate(4.441e-06, 0) | (q[0])
Dgate(0+1e+07j, 0) | (q[0])


In [ ]:
!pip install pennylane-sf

In [96]:
#@title Strawberryfields to Pennylane gate
import cmath
import pennylane as qml

def sf2pennylane(string):
  Beamsplitter = (string.startswith('BSgate'), (2, 2))
  ControlledAddition = (string.startswith('CXgate'), (1, 2))
  ControlledPhase = (string.startswith('CZgate'), (1, 2))
  CrossKerr = (string.startswith('CKgate'), (1, 2))
  CubicPhase = (string.startswith('Vgate'), (1, 1))
  Displacement = (string.startswith('Dgate'), (2, 1))
  Kerr = (string.startswith('Kgate'), (1, 1))
  QuadraticPhase = (string.startswith('Pgate'), (1, 1))
  Rotation = (string.startswith('Rgate'), (1, 1))
  Squeezing = (string.startswith('Sgate'), (2, 1)) 
  TwoModeSqueezing = (string.startswith('S2gate'), (2, 2))
  X = (string.startswith('MeasureX'), (0, 1))
  P = (string.startswith('MeasureP'), (0, 1))

  bool_list = [Beamsplitter, ControlledAddition, ControlledPhase,
               CrossKerr, CubicPhase, Displacement, Kerr, QuadraticPhase,
               Rotation, Squeezing, TwoModeSqueezing]
  
  gate_list = [qml.Beamsplitter, qml.ControlledAddition, qml.ControlledPhase,
               qml.CrossKerr, qml.CubicPhase, qml.Displacement,
               qml.Kerr, qml.QuadraticPhase, qml.Rotation,qml.Squeezing,
               qml.TwoModeSqueezing]

  for i, (b, num) in enumerate(bool_list):
    if b:
      gate = gate_list[i]
      
      if num[0] == 0:
        wires = int(string[string.find('[')+1:string.find(']')])
        return gate(wires=[wires])

      elif num[0] == 1:
        p = float(string[string.find('(')+1:string.find(')')])

        if num[1] == 1:
          wires = int(string[string.find('[')+1:string.find(']')])
          return gate(p, wires=[wires])

        elif num[1] == 2:
          wires = hamiltonian[1][hamiltonian[1].find('|')+2:]
          wire1 = wires[:wires.find(',')]
          wire1 = int(wire1[wire1.find('[')+1:wire1.find(']')])

          wires = hamiltonian[1][hamiltonian[1].find('|')+2:]
          wire2 = wires[wires.find(','):]
          wire2 = int(wire2[wire1.find('[')+1:wire2.find(']')])
          return gate(p, wires=[wire1, wire2])

        else:
          assert 0, f"Error not support the number of wires is {num[1]}."

      elif num[0] == 2:
        p = string[string.find('(')+1:string.find(')')]
        
        try:
          p1 = float(p[:p.find(',')])
          p2 = float(p[p.find(',')+2:])
        except:
          p1, p2 = cmath.polar(complex(p[:p.find(',')]))

        if num[1] == 1:
          wires = int(string[string.find('[')+1:string.find(']')])
          return gate(p1, p2, wires=[wires])

        elif num[1] == 2:
          wires = hamiltonian[1][hamiltonian[1].find('|')+2:]
          wire1 = wires[:wires.find(',')]
          wire1 = int(wire1[wire1.find('[')+1:wire1.find(']')])

          wires = hamiltonian[1][hamiltonian[1].find('|')+2:]
          wire2 = wires[wires.find(','):]
          wire2 = int(wire2[wire2.find('[')+1:wire2.find(']')])
          return gate(p1, p2, wires=[wire1, wire2])

        else:
          assert 0, f"Error not support the number of wires is {num[1]}."

  assert 0, "Error no gate support."


In [97]:
import pennylane as qml
from pennylane.optimize import GradientDescentOptimizer

In [98]:
num_wires = 1
dev = qml.device("strawberryfields.fock", wires=num_wires, cutoff_dim=20)

Create ansatz for guesting answer

In [99]:
def ansatz(var):
  for wire in range(num_wires):
    qml.Rotation(var[0+wire], wires=wire)
    qml.Squeezing(var[1+wire], 0.0, wires=wire)
    qml.Displacement(var[2+wire], 0.0, wires=wire)

@qml.qnode(dev)
def circuit(var):
  for v in var:
    ansatz(v)

  for ham in hamiltonian:
     sf2pennylane(ham)

  return qml.expval(qml.NumberOperator(wires=0))

@qml.qnode(dev)
def prediction(var):
  for v in var:
    ansatz(v)

  return qml.expval(qml.NumberOperator(wires=0))

In [100]:
np.random.seed(1)
num_layers = 1
var_init = 0.05*np.random.randn(num_layers, 3*num_wires)
var_init

array([[ 0.08121727, -0.03058782, -0.02640859]])

In [101]:
print(qml.draw(circuit)(var_init))

 0: ──R(0.0812)──S(-0.0306, 0)──D(-0.0264, 0)──R(0.785)──S(-0.001, 0)──R(-0.786)──D(4.44e-06, 0)──D(1e+07, 1.57)──┤ ⟨n⟩ 



Training with gradient descent

In [102]:
opt = GradientDescentOptimizer(0.01)

s = 0.1
alpha = np.pi/2

disc = 2
check_prev = 3
it = 0

var = var_init.copy()
loss_plot = []
break_count = 0


for it in range(1001):
    var, _cost = opt.step_and_cost(lambda v: circuit(v), var) #, grad_fn=lambda var: parameter_shift_cv(circuit, var) 
    
    loss_plot.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.00000000000 
Iter:   100 | Cost: 0.00000000000 
Iter:   200 | Cost: 0.00000000000 
Iter:   300 | Cost: 0.00000000000 
Iter:   400 | Cost: 0.00000000000 
Iter:   500 | Cost: 0.00000000000 
Iter:   600 | Cost: 0.00000000000 
Iter:   700 | Cost: 0.00000000000 
Iter:   800 | Cost: 0.00000000000 
Iter:   900 | Cost: 0.00000000000 
Iter:  1000 | Cost: 0.00000000000 


Result 

In [104]:
print("Target:", target)
print("Prediction: ", prediction(var))

Target: 1e-08
Prediction:  0.001633320101899182
